pip install torch torchvision torchaudio
pip install transformers datasets

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

/home/ggupte/practice/ds/deep_learning/venv-deep-learn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-19 12:29:46.557942: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 12:29:47.805756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 12:29:49.850268: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Load the IMDB dataset
dataset = load_dataset('imdb')
# Use only a subset for training to keep it quick
train_dataset = dataset['train'].shuffle(seed=42).select([i for i in range(1000)])
test_dataset = dataset['test'].shuffle(seed=42).select([i for i in range(1000)])

In [5]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a tokenization function
def tokenize_function(examples):
	return tokenizer(examples['text'], padding='max_length', truncation=True)

In [6]:
# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [7]:
#. Create Data Loaders
# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [8]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
pip install accelerate -U

     |████████████████████████████████| 314 kB 1.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
#Define Training Arguments and Trainer
#The Trainer class simplifies the training process significantly.
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
)

# Create a trainer
trainer = Trainer(
    model=model,                         # The instantiated 🤗 Transformers model to be trained
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset            # Evaluation dataset
)

In [ ]:
#8. Train the Model
# Train the model
trainer.train()
	
#9. Evaluate the Model
# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

In [ ]:
#10. Save the Model
# Save the fine-tuned model
model.save_pretrained('./sentiment_model')
tokenizer.save_pretrained('./sentiment_tokenizer')

In [ ]:
#11. Load the Model for Inference
#	For making predictions on new data, you can load the model and tokenizer.
	
# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./sentiment_model')
tokenizer = BertTokenizer.from_pretrained('./sentiment_tokenizer')

# Function to predict sentiment
def predict_sentiment(text):
	inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
	with torch.no_grad():
		outputs = model(**inputs)
		logits = outputs.logits
		prediction = torch.argmax(logits, dim=-1).item()
		return "positive" if prediction == 1 else "negative"

	# Example prediction
	print(predict_sentiment("This movie was amazing!"))
	print(predict_sentiment("I did not like this movie at all."))
	

12. outputs.logits
	Logits: These are the raw scores produced by the model before applying any activation function like softmax. They represent the model's confidence in each class (in this case, sentiment classes)	
	
13. outputs = model(**inputs)	
	Model: The model is a pre-trained transformer model loaded from the Hugging Face library.
	Inputs: The **inputs syntax unpacks the dictionary of inputs, which typically includes input_ids and attention_mask, and passes them to the model for inference.
	Outputs: The model returns an object that includes various outputs, with logits being the raw, unnormalized scores for each class.
	
14. Predicting the Sentiment:
	prediction = torch.argmax(logits, dim=-1).item()
	
	torch.argmax: Finds the index of the highest value in logits, which corresponds to the predicted class. dim=-1 specifies that the operation is performed across the last dimension, which is the class dimension.
	item(): Converts the PyTorch tensor to a standard Python integer.